# Tracking and processing

## Overview

In order to track and process the trial, you will now need to run another set script. This will involve the following steps:
1. Import the relevant libraries from `cv-tracer`
2. Re-open the appropriate `Trial` object.
3. Enter parameters relevant to this run, and initialize a `CVTracer` object using those parameters. 
4. Run the tracking loop.
5. Process the tracks to convert pixels to centimeters and calculate kinematic quantities like velocity, turning velocity, and acceleration.

## Running the script

First we can import the relevant libraries from `numpy` and `cv-tracer`.

In [1]:
#!/usr/bin/python3
import sys
cvhome="/home/apatch/Code/cv-tracer"
sys.path.insert(0, cvhome)
import argparse
from TrAQ.Trial import Trial
from TrAQ.CVTracer import CVTracer
import numpy as np

Next we will re-open the trial we initialized in the last step and input appropriate parameters.

In [ ]:
def arg_parse():
    parser = argparse.ArgumentParser(description="OpenCV2 Fish Tracker")
    parser.add_argument("raw_video", type=str, help="path to raw video")
    parser.add_argument("-ts","--t_start", type=float, help="start time, in seconds", default=0)
    parser.add_argument("-te","--t_end", type=float, help="end time, in seconds", default=-1)
    parser.add_argument("-npix","--n_pixel_blur", type=float, help="square-root of n-pixels for threshold blurring", default=3)
    parser.add_argument("-bs","--block_size", type=int, help="contour block size",default=15)
    parser.add_argument("-th","--thresh_offset", type=int, help="threshold offset for contour-finding",default=13) 
    parser.add_argument("-amin","--min_area", type=int, help="minimum area for threhold detection", default=20)
    parser.add_argument("-amax","--max_area", type=int, help="maximum area for threhold detection", default=500)
    parser.add_argument("-lt","--trail_length", type=int, help="length of trail", default=3)
    parser.add_argument("-on","--online_viewer", help="view tracking results in real time", action='store_true') 
    parser.add_argument("-vs","--view_scale", help="to scale size of window for online viewing", default=1.) 
    parser.add_argument("-RGB", help="generate movie in RGB (default is greyscale)", action='store_true')
    parser.add_argument("-GPU", help="use UMat file handling for OpenCV Transparent API", action='store_true') 
    #parser.add_argument("-AMM", help="switch to mean adaptive method instead of Gaussian", action='store_true') 
    #parser.add_argument("-TI","--thresh_invert", help="Invert adaptive threshold", action='store_true') 
    return parser.parse_args()

args = arg_parse()
# open up the Trial
trial       = Trial(args.raw_video)
# initialize a CVTracer
frame_start = int(args.t_start * trial.fps)
frame_end   = int(args.t_end   * trial.fps)

Now we can initialize the `CVTracer` object. This object will allow us to step through each frame in a `Trial` video by using functions written to operate on `CVTracer` objects.

In [ ]:
cvt         = CVTracer( trial, frame_start = frame_start, frame_end = frame_end, 
                        n_pixel_blur = args.n_pixel_blur, block_size = args.block_size, 
                        threshold_offset = int(args.thresh_offset), 
                        min_area = args.min_area, max_area = args.max_area, 
                        len_trail = args.trail_length, 
                        RGB = args.RGB, online = args.online_viewer, view_scale = args.view_scale, 
                        GPU = args.GPU ) 

cvt.print_title()

Now that we have initialized the `CVTracer` object `cvt`, we can run through each relevant frame and perform the necessary steps of tracking. 

For a deeper look at each of these functions, you can open up `cv-tracer/TrAQ/CVTracer.py`. Careful not to edit unless you know what you're doing. 

In [ ]:
cvt.set_frame(frame_start)
for i_frame in range(cvt.frame_start, cvt.frame_end + 1):
    # load next frame into video tracer
    if cvt.get_frame():
        # first remove all information from outside of tank
        cvt.mask_tank()       
        # detect contours in current frame
        cvt.detect_contours()
        # analyze contours for center and long axis
        cvt.analyze_contours()
        # estimate connection between last frame and current frame
        cvt.connect_frames()
        # store results from current frame in the trial object
        cvt.update_trial()
        # write frame with traces to new video file, show if online
        cvt.draw()
        # write frame to video output
        cvt.write_frame()
        # post frame to screen, may accept user input to break loop
        if not cvt.post_frame(): break
        # print time to terminal
        cvt.print_current_frame()
# put things away
cvt.release()
cvt.trial.save()

Finally, we will convert the pixel data to centimeters and calculate some kinematic quantities. The `Trial` object knows how to convert to centimeters thanks to the radius we entered when we defined the `Tank`. It can convert frames to time using the frames per second we gave upon defining the `Trial`. With these two conversions, we are able to determine quantites like velocity and acceleration and director turning velocity and acceleration.

In [ ]:
# first convert pixels to centimeters
trial.convert_pixels_to_cm()

#trial.transform_lens()
trial.calculate_kinematics()
trial.save()

Now that you have completed this stage, you should check to see if the tracking worked. You can do this by opening `traced.mp4`, which should show the contours of the fish. There will still be errors, but the important thing is to ensure that the fish are tracked in most frames and in all portions of the arena. If not, please email apatch@fau.edu.